In [1]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=0)
import joblib


In [2]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/bandai_l2p_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/bandai_l2a_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/bandai_l2d_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/S_BANDAI_5frame.pkl') 

In [3]:
dataset = pd.read_csv('datasets/BANDAI_5frame.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.433596,0.130036,0.335475,0.538619,0.184473,0.310089,0.210679,0.093915,0.236151,0.223894,...,-0.137483,-0.139467,0.092920,0.196129,0.221155,0.137483,0.139467,0.05,-0.4,0.0
1,0.348051,0.145184,0.256784,0.455501,0.197954,0.284215,0.172405,0.104876,0.236151,0.223894,...,-0.072441,-0.095084,0.048935,0.032804,0.070625,0.072096,0.045111,0.05,-0.4,0.0
2,0.320294,0.206306,0.180224,0.380190,0.234406,0.259905,0.133006,0.127385,0.236151,0.223894,...,-0.039016,-0.093423,0.030819,0.043059,0.028182,0.036532,0.014232,0.05,-0.4,0.0
3,0.400389,0.287300,0.136974,0.319861,0.279208,0.239450,0.112632,0.156856,0.236151,0.223894,...,-0.036825,-0.101489,0.044534,0.054382,0.015679,0.012813,0.021506,0.05,-0.4,0.0
4,0.465923,0.356129,0.136171,0.309995,0.313229,0.224432,0.112950,0.186680,0.236151,0.223894,...,-0.054338,-0.117573,0.044670,0.036618,0.020074,0.022012,0.024961,0.05,-0.4,0.0


In [4]:
train_dataset = dataset.sample(frac=0.95, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 74623
No Test Samples: 3928


In [5]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [6]:
train_emotions_p = model_p.predict(train_dataset)
train_emotions_a = model_a.predict(train_dataset)
train_emotions_d = model_d.predict(train_dataset)

rows = []
for i in range(len(train_dataset)):
    rows.append([train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]])

train_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.125231,-0.347150,-0.073132
1,0.204304,-0.067318,0.048539
2,0.158746,-0.327179,-0.099823
3,0.098928,-0.321751,-0.036743
4,0.173503,-0.123865,-0.068499


In [7]:
test_emotions_p = model_p.predict(test_dataset)
test_emotions_a = model_a.predict(test_dataset)
test_emotions_d = model_d.predict(test_dataset)

rows = []
for i in range(len(test_dataset)):
    rows.append([test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]])

test_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.112296,-0.353002,-0.039603
1,0.161262,-0.240276,-0.053446
2,0.156622,-0.209693,-0.012130
3,0.124465,-0.365829,-0.083854
4,0.168878,-0.243771,-0.059709


In [8]:
def vae_loss(input_vols, output_vols):
    beta = 1e-7
    kl_loss = K.sum(-1 - K.log(K.exp(log_var)) + K.exp(log_var) + K.square(mu))/2
    return K.mean((input_vols-output_vols)**2) + beta*kl_loss

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

## Sampling Layer

In [31]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Encoder

In [32]:
latent_dim = 5

encoder_inputs = keras.Input(shape=(25, ))
x = layers.Dense(15, activation="relu")(encoder_inputs)
x = layers.Dense(10, activation="relu")(x)
x = layers.Dense(5, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 25)]         0           []                               
                                                                                                  
 dense_14 (Dense)               (None, 15)           390         ['input_5[0][0]']                
                                                                                                  
 dense_15 (Dense)               (None, 10)           160         ['dense_14[0][0]']               
                                                                                                  
 dense_16 (Dense)               (None, 5)            55          ['dense_15[0][0]']               
                                                                                            

## Decoder

In [33]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(10, activation="relu")(latent_inputs)
x = layers.Dense(15, activation="relu")(x)
x = layers.Dense(20, activation="relu")(x)

decoder_outputs = layers.Dense((25, )[0], activation="linear")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense_17 (Dense)            (None, 10)                60        
                                                                 
 dense_18 (Dense)            (None, 15)                165       
                                                                 
 dense_19 (Dense)            (None, 20)                320       
                                                                 
 dense_20 (Dense)            (None, 25)                525       
                                                                 
Total params: 1,070
Trainable params: 1,070
Non-trainable params: 0
_________________________________________________________________


## VAE

In [34]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            
            reconstruction = self.decoder(z)
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = (100 * reconstruction_loss) + (kl_loss)
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

## Train

In [35]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(train_dataset, epochs=64, batch_size=32)

Epoch 1/64
2332/2332 [==============================] - 4s 1ms/step - loss: 1.6972 - reconstruction_loss: 0.0096 - kl_loss: 0.0041
Epoch 2/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7513 - reconstruction_loss: 0.0075 - kl_loss: 7.2889e-06
Epoch 3/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7450 - reconstruction_loss: 0.0075 - kl_loss: 4.0257e-06
Epoch 4/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7451 - reconstruction_loss: 0.0075 - kl_loss: 3.8303e-06
Epoch 5/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7478 - reconstruction_loss: 0.0075 - kl_loss: 3.8174e-06
Epoch 6/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7459 - reconstruction_loss: 0.0075 - kl_loss: 1.8254e-06
Epoch 7/64
2332/2332 [==============================] - 3s 1ms/step - loss: 0.7462 - reconstruction_loss: 0.0075 - kl_loss: 1.1270e-06
Epoch 8/64
2332/2332 [==============================] - 3s 

KeyboardInterrupt: 

## Test

In [ ]:
conv_dict = {
    (0.05, -0.4, 0.0): "neutral",
    (0.15, -0.7, -0.2): "tired",
    (-0.1, -0.55, -0.15): "exhausted",
    (0.2, -0.75, -0.3): "exhausted_2",
    (-0.5, 0.8, 0.9): "angry",
    (0.8, 0.5, 0.15): "happy",
    (0.6, 0.4, 0.1): "happy_2",
    (-0.6, -0.4, -0.3): "sad",
    (0.4, 0.2, 0.3): "proud", 
    (0.3, 0.3, 0.9): "confident", 
    (0.25, 0.15, 0.3): "confident_2",
    (0.25, 0.15, 0.3): "confident_3",
    (0.3, 0.4, 0.6): "confident_4", 
    (-0.6, 0.7, -0.8): "afraid",
    (0.1, 0.6, 0.4): "active", 
}

colour_dict = {
    "neutral": "crimson",
    "tired": "springgreen",
    "exhausted": "cornflowerblue",
    "exhausted_2": "darkorange"  ,
    "angry": "gold",
    "happy": "olive",
    "happy_2": "lightseagreen",
    "sad": "plum",
    "proud": "chocolate",
    "confident": "olivedrab",
    "confident_2": "purple",
    "confident_4": "lightslategray",
    "afraid": "lightpink", 
    "active": "slateblue", 
}

def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    
    actual_labels = []
    for i in range(len(labels)):
        point_coords = (labels.iloc[i][0], labels.iloc[i][1], labels.iloc[i][2])

        actual_labels.append(colour_dict[conv_dict[point_coords]])

    
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=actual_labels)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

plot_label_clusters(vae, test_dataset, test_emotions_OG)


In [26]:
sample = np.asarray(test_dataset.iloc[1])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.6066639   0.25030951  0.36207177  0.44697082  0.30988349  0.34038932
   0.18311099  0.1162812   0.23615099  0.22389402  0.26641542  0.09919871
   0.06424907  0.10578747  0.16329062 -0.10280018 -0.06087656 -0.0186052
  -0.08945924 -0.07371857  0.01320025  0.02030674  0.01414669  0.01907025
   0.01265828]]


In [27]:
mean, var, generated = encoder.predict(sample)
print(generated)

[[-1.0225942   0.29392263  0.07719255  1.0344585  -1.0845417 ]]


In [28]:
regen = decoder.predict(generated)
print(regen)

[[ 0.5506572   0.32379323  0.31053263  0.4553539   0.34142768  0.34105572
   0.23057324  0.22952063  0.23614955  0.22389574  0.345175    0.08628955
   0.08113877  0.09854872  0.14678863 -0.09339849 -0.09299247 -0.09039417
  -0.08972329 -0.08726535  0.01823894  0.01915274  0.02006095  0.01667385
   0.01130016]]


In [29]:
scaled_sample = scaler.transform(sample)

real_coordinates = (
    model_p.predict(scaled_sample),
    model_a.predict(scaled_sample),
    model_d.predict(scaled_sample)
)

scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % np.asarray(generated_coordinates))

Real: [[0.6026331 ]
 [0.42717165]
 [0.11683352]]
Predicted: [[ 0.1870146 ]
 [-0.3148598 ]
 [-0.14555968]]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
